In [1]:
import os
import sys
sys.path.append('/kaggle/input/timm-0-6-9/pytorch-image-models-master')
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
# import cv2
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from helper import *

In [2]:
%%time
helmet_data= pd.read_csv('./train_baseline_helmets.csv')
tracking_data = pd.read_csv('./train_player_tracking.csv')
labels = pd.read_csv('./train_labels.csv')

Wall time: 5.48 s


In [3]:
print(f'this is the train helment detector: {helmet_data.shape}')
print(f'this is the train tracking data: {tracking_data.shape}')
print(f'this is the train labels: {labels.shape}')

this is the train helment detector: (3783616, 12)
this is the train tracking data: (1353053, 17)
this is the train labels: (4721618, 7)


In [6]:
%%time
label_exp = expand_contact_id(labels)
use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa',
    'team', 'position'
]
combined_df = create_features(label_exp, tracking_data, use_cols=use_cols)

Wall time: 39.5 s


In [7]:
combined_df.shape

(660553, 41)

In [ ]:
combined_df.to_csv('./combined_df.csv')

In [2]:
%%time
combined_df = pd.read_csv('./combined_df.csv')

Wall time: 1.97 s


In [6]:
helmet_data['frame'].min()

289

In [10]:
def add_bboxes_details(frame, player1, player2, game_play, view):
    '''
    takes each row in comined_df and adds the required bbox information
    (may need more processing to convert to correct datatype)
    '''
    bboxes = []
    window = 24
    query = f"frame >= {frame-window}  and frame <= {frame+window} and game_play == '{game_play}'"
    if player2 == 'G':
        query += f"  and nfl_player_id in ({player1}, 'G')"
    else:
        query += f"  and nfl_player_id in ({player1}, {player2})"
    filt = helmet_data.query(query)

    tmp = filt[filt["view"] == view]
    tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
    tmp['centre_x'] = tmp['left'] + tmp['width']/2
    tmp['centre_y'] = tmp['top'] + tmp['height']/2
    frame_list = tmp.index # these are the available frames
    for fr in range(frame - window, frame + window + 1):
        if fr in frame_list:
            x, y = tmp.loc[fr][['centre_x','centre_y']]
            bboxes.append([x, y])
        else:
            bboxes.append([np.nan, np.nan])
    return np.array(bboxes)
   

In [12]:
combined_df.loc[0]

contact_id              58168_003392_0_37084_38567
game_play                             58168_003392
datetime                  2020-09-11T03:01:48.100Z
step                                             0
nfl_player_id_1                              37084
nfl_player_id_2                              38567
contact                                          0
x_position_1                                  41.9
y_position_1                                 20.08
speed_1                                       0.54
distance_1                                    0.06
direction_1                                 252.69
orientation_1                               262.31
acceleration_1                                0.92
sa_1                                           0.9
team_1                                        away
position_1                                      DE
x_position_2                                 40.37
y_position_2                                 19.88
speed_2                        

In [11]:
%%time
temp = combined_df.loc[:100]
temp['bbox_endzone'] = temp.apply(
    lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Endzone'),
    axis=1
)
temp

Wall time: 16.9 s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,x_position_1,y_position_1,speed_1,...,rel_speed_y,rel_speed_mag,rel_speed_ori,rel_acceleration_x,rel_acceleration_y,rel_acceleration_mag,rel_acceleration_ori,G_flug,frame,bbox_endzone
0,58168_003392_0_37084_38567,58168_003392,2020-09-11T03:01:48.100Z,0,37084,38567,0,41.90,20.08,0.54,...,0.319658,1.019588,18.271264,1.495569,0.381257,1.543400,14.301488,False,300,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
1,58168_003392_0_37084_G,58168_003392,2020-09-11T03:01:48.100Z,0,37084,G,0,41.90,20.08,0.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,300,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
2,58168_003392_0_37211_46445,58168_003392,2020-09-11T03:01:48.100Z,0,37211,46445,0,39.59,17.07,0.53,...,0.568473,0.599983,71.348359,0.033670,0.730436,0.731212,87.360703,False,300,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
3,58168_003392_0_37211_G,58168_003392,2020-09-11T03:01:48.100Z,0,37211,G,0,39.59,17.07,0.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,300,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
4,58168_003392_0_38556_G,58168_003392,2020-09-11T03:01:48.100Z,0,38556,G,0,41.93,30.61,0.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,300,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,58168_003392_2_46137_G,58168_003392,2020-09-11T03:01:48.300Z,2,46137,G,0,49.14,22.94,2.58,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,312,"[[nan, nan], [nan, nan], [600.5, 363.5], [602...."
97,58168_003392_2_46445_G,58168_003392,2020-09-11T03:01:48.300Z,2,46445,G,0,40.47,17.85,1.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,312,"[[nan, nan], [nan, nan], [256.5, 312.0], [256...."
98,58168_003392_2_46522_G,58168_003392,2020-09-11T03:01:48.300Z,2,46522,G,0,40.62,38.78,1.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,312,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
99,58168_003392_2_47837_G,58168_003392,2020-09-11T03:01:48.300Z,2,47837,G,0,43.96,38.70,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,312,"[[nan, nan], [nan, nan], [nan, nan], [nan, nan..."
